In [1]:
##VEST Louisiana 2018 Shapefile validation

"""
Louisiana
---------
Election results from LA Secretary of State (https://voterportal.sos.la.gov/static/)
Precinct shapefile from LA House of Representatives (http://house.louisiana.gov/H_Redistricting2011/default_LouisianaPrecinctShapefiles)

Early votes were reported at the parish level. These were distributed by candidate to precincts based on their share of the precinct-level reported vote.

Election results from the following parishes include "alpha" precincts in which voters within the same geographic boundaries are assigned to separate precincts by the first letter of their surname: Ascension, Assumption, Bossier, Caddo, East Baton Rouge, Lafayette, Lafourche, Rapides, St. Charles, St. Landry, Terrebonne

The following precincts were modified to reflect alterations enacted prior to the 2018 election:

Avoyelles: Merge 2-5B/6-1A
Plaquemines: Merge 2-1/2-2, 4-1/4-2, 5-1/5-2
St. Charles: Merge 2-6/2-7, 3-1/3-6, 3-3/3-4, 6-2/6-3, 6-4/6-5
Vermilion: Split 49B-1/49B-2
West Baton Rouge: Split 2-A/2-B; 11-A/11-B

G18SOSRARD - Kyle Ardoin (Republican Party)
G18SOSREDM - "Rick" Edmongs (Republican Party)
G18SOSRSTO - Julie Stokes (Republican Party)
G18SOSRKEN - Thomas J. Kennedy III (Republican Party)
G18SOSRCRO - A.G. Crowe (Republican Party)
G18SOSRCLO - Heather Cloud (Republican Party)
G18SOSDCOL - "Gwen" Collins-Greenup (Democratic Party)
G18SOSDFRE - Renee Fontenot Free (Democratic Party)
G18SOSNMOR - Matthew Paul "Matt" Moreau (No Party)

R18SOSRARD - Kyle Ardoin (Republican Party)
R18SOSDCOL - "Gwen" Collins-Greenup (Democratic Party)


Raw files: 
    - 2018 Louisiana Precinct File (from LA House of Representatives, as a .shp)
    - 2018 Louisiana SoS Primary Election results - precinct (from LA SoS, as a .csv)
    - 2018 Louisiana SoS Priary Election results - parish (from LA SoS, as a .csv)
    - 2018 Louisiana SoS General Election results (from LA SoS, as a .csv)
    - 2018 Louisiana SoS General Election results (from LA SoS, as a .csv)
    
    
Steps:
1. Load in all four files (one final, five raw)
2. Merge listed alterations above in raw file, rename accordingly
3. Validate the geographies with each other (the merged raw and the final)
MERGE???
4. Election results alpha precincts???
5. Interpolate precinct results from county results (mail in and early)
6. 
"""

'\nLouisiana\n---------\nElection results from LA Secretary of State (https://voterportal.sos.la.gov/static/)\nPrecinct shapefile from LA House of Representatives (http://house.louisiana.gov/H_Redistricting2011/default_LouisianaPrecinctShapefiles)\n\nEarly votes were reported at the parish level. These were distributed by candidate to precincts based on their share of the precinct-level reported vote.\n\nElection results from the following parishes include "alpha" precincts in which voters within the same geographic boundaries are assigned to separate precincts by the first letter of their surname: Ascension, Assumption, Bossier, Caddo, East Baton Rouge, Lafayette, Lafourche, Rapides, St. Charles, St. Landry, Terrebonne\n\nThe following precincts were modified to reflect alterations enacted prior to the 2018 election:\n\nAvoyelles: Merge 2-5B/6-1A\nPlaquemines: Merge 2-1/2-2, 4-1/4-2, 5-1/5-2\nSt. Charles: Merge 2-6/2-7, 3-1/3-6, 3-3/3-4, 6-2/6-3, 6-4/6-5\nVermilion: Split 49B-1/49B-2\

In [2]:
"""Run all of the necessary imports"""
import matplotlib.pyplot as plt # for plotting maps
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
from statistics import mean, median
import timeit 
import requests

In [3]:
""""Start a timer"""
start = timeit.default_timer()

In [4]:
"""Import the four raw csv datasets"""

SoSGen_Parish = pd.read_csv('./raw-data/election_results_la/SoSGeneral2018_LAParsih.csv')
SoSGen_Precinct = pd.read_csv('./raw-data/election_results_la/SoSGeneral2018_LAPrecincts.csv')
SoSPrim_Parish = pd.read_csv('./raw-data/election_results_la/SoSPrimary2018_LAParish.csv')
SoSPrim_Precinct = pd.read_csv('./raw-data/election_results_la/SoSPrimary2018_LAPrecincts.csv')


In [5]:
"""Import the two shapefiles (one raw, one final)"""

precincts = gp.read_file('./raw-data/2018 LA Precincts for the Web/2018 LA Precincts for the Web.shp')
final = gp.read_file('./final_vest_la_2018/la_2018.shp')

In [6]:
"""In order to merge/dissolve the boundaries listed above in the documentation,
we need to identify which those are -- this only done by county so 
we will retrieve the county FIPS for those counties
"""
#Function to retrieve the county fips codes for all counies
def counties(state_fips):
    """Inputs: state fips code
    Process: Retrieves a list of counties in the given state from the Census API.  
    Outputs: A list of county fips codes in the state. """
    #uses the fips input into the census api
    resp = requests.get(
        "https://api.census.gov/data/2010/dec/sf1"
        "?get=NAME&for=county:*&in=state:{}".format(state_fips)  #uses the fips input to locate the state
    )
    #retrieves the data as a json 
    test = resp.json()
    header, *rows = resp.json()
    #county column is "county"
    county_column_index = header.index("county")
    county_fips = set(row[county_column_index] for row in rows) #sequence of counties 
    county_names_index = header.index("NAME")
    county_names = set(row[county_names_index] for row in rows) #names of the counties
    county_fips = np.array(list(county_fips)) #make the sets into numpy arrays
    county_names = np.array(list(county_names))
    df = pd.DataFrame({'COUNTYFP10': county_fips, 'COUNTYNAMES': county_names}) #make pd dataframe of arrays
    return df  #returns the fips codes of all counties

#run the function for LA (FIPS code 22)
LA_counties = counties(22)

#check the number of parishes and values for the columns
LA_counties.nunique()

COUNTYFP10     64
COUNTYNAMES    64
dtype: int64

In [7]:
#check the number of values for counties in the precincts file
precincts['COUNTYFP10'].nunique()

64

In [8]:
##merge the county names to the precincts shapefile by the fips
precincts_countynames = precincts.merge(LA_counties, on=['COUNTYFP10'], how='inner')
#make sure it's still a geodataframe
type(precincts_countynames)

geopandas.geodataframe.GeoDataFrame

In [9]:
##Check to make sure no precincts were dropped in the merge
precincts_cases = precincts.shape[0]
prec_names = precincts_countynames.shape[0]
print("number of differing precincts: ", abs(precincts_cases-prec_names))


number of differing precincts:  0


In [10]:
final['GEOID10']=final['STATEFP10']+final['COUNTYFP10']+final['VTDST10']
final=final.merge(LA_counties,on=['COUNTYFP10'], how='inner')
joined = pd.merge(final, precincts_countynames, on=['GEOID10'], indicator=True, how='outer')
differing_precincts = joined[joined['_merge'] != 'both']
print('number of differing precincts: ', differing_precincts.shape[0])
differing_precincts_GEOIDs = [differing_precincts['GEOID10']]
print(differing_precincts_GEOIDs)
joined_inner = pd.merge(final, precincts_countynames, on=['GEOID10'], how='inner')
final_rows = final.shape[0]
precinct_rows = precincts_countynames.shape[0]
print("Number of different rows: ", abs(final_rows-precinct_rows))
print("raw file rows: ", precinct_rows)
print("vest-la rows: ", final_rows)
print("outer merge test rows: ", joined.shape[0])
differing_precincts.head(14)

number of differing precincts:  14
[82      2211349B-2
2985       221212A
2986      2212111B
3327       2208751
3710     220092-5B
3711      220752-2
3712      220755-2
3713      220754-2
3714      220896-3
3715      220896-5
3716      220893-6
3717      220892-7
3718      220893-4
3719       2208758
Name: GEOID10, dtype: object]
Number of different rows:  6
raw file rows:  3716
vest-la rows:  3710
outer merge test rows:  3720


,STATEFP10_x,COUNTYFP10_x,VTDST10_x,NAME10_x,G18SOSRARD,G18SOSREDM,G18SOSRSTO,G18SOSRKEN,G18SOSRCRO,G18SOSRCLO,...,VTD,TRACT,BGROUP,BLOCK,TOT_POP,SHAPE_AREA,SHAPE_LEN,geometry_y,COUNTYNAMES_y,_merge
82,22,113,49B-2,Precinct 49B-2,54.0,21.0,19.0,21.0,12.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,left_only
2985,22,121,2A,Precinct 2A,310.0,119.0,72.0,38.0,14.0,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,left_only
2986,22,121,11B,Precinct 11B,20.0,11.0,4.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,left_only
3327,22,087,51,Precinct 51,13.0,17.0,26.0,21.0,69.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,left_only
3710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2-5B,None,None,None,75.0,0.00016,0.072198,"POLYGON ((-91.89467 31.11142, -91.89445 31.111...","Plaquemines Parish, Louisiana",right_only
3711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2-2,None,None,None,1393.0,0.00048,0.097914,"POLYGON ((-89.99418 29.89268, -89.99420 29.892...","Richland Parish, Louisiana",right_only
3712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5-2,None,None,None,1614.0,0.00067,0.205141,"POLYGON ((-90.01772 29.87860, -90.01763 29.878...","Richland Parish, Louisiana",right_only
3713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4-2,None,None,None,1833.0,0.00052,0.134172,"POLYGON ((-89.99002 29.83905, -89.99057 29.838...","Richland Parish, Louisiana",right_only
3714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6-3,None,None,None,1889.0,0.00249,0.228105,"POLYGON ((-90.39931 30.07477, -90.39698 30.073...","Morehouse Parish, Louisiana",right_only
3715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6-5,None,None,None,352.0,0.00236,0.283257,"POLYGON ((-90.37115 30.05746, -90.37097 30.057...","Morehouse Parish, Louisiana",right_only


In [11]:
"""Avoyelles: Merge 2-5B/6-1A
St. Charles: Merge 2-6/2-7, 3-1/3-6, 3-3/3-4, 6-2/6-3, 6-4/6-5
Vermilion: Split 49B-1/49B-2
West Baton Rouge: Split 2-A/2-B; 11-A/11-B"""

'Avoyelles: Merge 2-5B/6-1A\nSt. Charles: Merge 2-6/2-7, 3-1/3-6, 3-3/3-4, 6-2/6-3, 6-4/6-5\nVermilion: Split 49B-1/49B-2\nWest Baton Rouge: Split 2-A/2-B; 11-A/11-B'

In [12]:
def dissolve_precinctsLA(countyname, precinct1, precinct2):
    data = precincts_countynames[(precincts_countynames.COUNTYNAMES == countyname) & ((precincts_countynames.VTD == precinct1) |  (precincts_countynames.VTD == precinct2))]
    dissolved_precinct = data.dissolve(by='COUNTYFP10', aggfunc='sum')
    return dissolved_precinct

In [13]:
PlaqueminesD1 = dissolve_precinctsLA('Plaquemines Parish, Louisiana', '2-1', '2-2')
PlaqueminesD2 = dissolve_precinctsLA('Plaquemines Parish, Louisiana', '4-1', '4-2')
PlaqueminesD3 = dissolve_precinctsLA('Plaquemines Parish, Louisiana', '5-1', '5-2')

In [14]:
"""acadia_raw = precincts_countynames[(precincts_countynames.COUNTYNAMES == 'Acadia Parish, Louisiana')]
acadia_elections = SoSGen_Precinct[(SoSGen_Precinct.Parish == 'Acadia')]
acadia_final = final[(final.COUNTYFP10 == '065')]
acadia_raw_cases = acadia_raw.shape[0]
acadia_elec_cases = acadia_elections.shape[0]
acadia_final_cases = acadia_final.shape[0]
print("raw: ", acadia_raw_cases, "SoS: ", acadia_elec_cases, "VEST: ", acadia_final_cases)
#acadia_elections['Precinct'] = acadia_elections['Precinct'].str[:2]
##combine ward and precinct
acadia_elections['ward_prec']=acadia_elections['Ward'].str.cat(acadia_elections['Precinct'],sep="-")
acadia_elections['ward_prec']"""

'acadia_raw = precincts_countynames[(precincts_countynames.COUNTYNAMES == \'Acadia Parish, Louisiana\')]\nacadia_elections = SoSGen_Precinct[(SoSGen_Precinct.Parish == \'Acadia\')]\nacadia_final = final[(final.COUNTYFP10 == \'065\')]\nacadia_raw_cases = acadia_raw.shape[0]\nacadia_elec_cases = acadia_elections.shape[0]\nacadia_final_cases = acadia_final.shape[0]\nprint("raw: ", acadia_raw_cases, "SoS: ", acadia_elec_cases, "VEST: ", acadia_final_cases)\n#acadia_elections[\'Precinct\'] = acadia_elections[\'Precinct\'].str[:2]\n##combine ward and precinct\nacadia_elections[\'ward_prec\']=acadia_elections[\'Ward\'].str.cat(acadia_elections[\'Precinct\'],sep="-")\nacadia_elections[\'ward_prec\']'

In [15]:
final.head(50)

,STATEFP10,COUNTYFP10,VTDST10,NAME10,G18SOSRARD,G18SOSREDM,G18SOSRSTO,G18SOSRKEN,G18SOSRCRO,G18SOSRCLO,G18SOSDCOL,G18SOSDFRE,G18SOSNMOR,R18SOSRARD,R18SOSDCOL,geometry,GEOID10,COUNTYNAMES
0,22,111,24,Precinct 24,89,49,25,45,11,12,60,53,3,80,13,"POLYGON ((-92.34486 33.01065, -92.34482 33.010...",2211124,"Lincoln Parish, Louisiana"
1,22,111,22,Precinct 22,149,78,30,79,21,30,11,15,5,110,13,"POLYGON ((-92.31067 32.84022, -92.31052 32.840...",2211122,"Lincoln Parish, Louisiana"
2,22,111,18,Precinct 18,124,71,22,77,11,23,7,15,1,51,0,"POLYGON ((-92.34032 32.70968, -92.33913 32.710...",2211118,"Lincoln Parish, Louisiana"
3,22,111,23,Precinct 23,100,60,42,93,17,17,7,15,3,98,17,"POLYGON ((-92.28528 32.71418, -92.28545 32.714...",2211123,"Lincoln Parish, Louisiana"
4,22,111,20,Precinct 20,93,58,20,67,4,21,8,9,5,84,3,"POLYGON ((-92.29989 32.68925, -92.30001 32.689...",2211120,"Lincoln Parish, Louisiana"
5,22,111,4,Precinct 4,119,90,29,99,16,31,31,80,6,101,25,"POLYGON ((-92.69172 33.01435, -92.69096 33.014...",221114,"Lincoln Parish, Louisiana"
6,22,111,8,Precinct 8,18,11,6,15,6,5,1,10,1,9,3,"POLYGON ((-92.48069 32.93496, -92.48071 32.934...",221118,"Lincoln Parish, Louisiana"
7,22,111,5,Precinct 5,51,52,17,47,17,13,23,24,3,62,15,"POLYGON ((-92.73405 32.80476, -92.73405 32.806...",221115,"Lincoln Parish, Louisiana"
8,22,111,6,Precinct 6,113,64,36,66,14,29,22,20,1,70,10,"POLYGON ((-92.61335 32.82448, -92.61293 32.824...",221116,"Lincoln Parish, Louisiana"
9,22,111,12,Precinct 12,29,26,4,22,8,11,123,74,1,31,106,"POLYGON ((-92.41456 32.75713, -92.41453 32.757...",2211112,"Lincoln Parish, Louisiana"


In [16]:
precincts.head(50)

,ID,AREA,OBJECTID,STATEFP10,COUNTYFP10,VTDST10,GEOID10,VTDI10,NAME10,NAMELSAD10,...,MCD,PLACE,VTD,TRACT,BGROUP,BLOCK,TOT_POP,SHAPE_AREA,SHAPE_LEN,geometry
0,3162,147.168381,3360,22,111,24,2211124,A,Precinct 24,Precinct 24,...,None,None,24,None,None,None,961,0.036730,1.254069,"POLYGON ((-92.34486 33.01065, -92.34482 33.010..."
1,2161,239.121475,3396,22,113,23,2211323,A,Precinct 23,Precinct 23,...,None,None,23,None,None,None,1175,0.057650,2.019706,"POLYGON ((-92.04562 29.50581, -92.04734 29.506..."
2,2148,392.914185,3371,22,113,16,2211316,A,Precinct 16,Precinct 16,...,None,None,16,None,None,None,166,0.094700,1.493221,"POLYGON ((-92.25397 29.80451, -92.24874 29.802..."
3,2987,29.314260,3352,22,111,22,2211122,A,Precinct 22,Precinct 22,...,None,None,22,None,None,None,1090,0.007300,0.574391,"POLYGON ((-92.31067 32.84022, -92.31052 32.840..."
4,3288,202.215469,3401,22,113,14,2211314,A,Precinct 14,Precinct 14,...,None,None,14,None,None,None,2403,0.048840,1.063200,"POLYGON ((-92.22548 29.94786, -92.22548 29.947..."
5,2544,32.090931,3338,22,111,18,2211118,A,Precinct 18,Precinct 18,...,None,None,18,None,None,None,963,0.007980,0.605118,"POLYGON ((-92.34032 32.70968, -92.33913 32.710..."
6,2364,34.426968,3347,22,111,23,2211123,A,Precinct 23,Precinct 23,...,None,None,23,None,None,None,895,0.008580,0.451130,"POLYGON ((-92.28528 32.71418, -92.28545 32.714..."
7,2311,5.970530,3384,22,113,20B-1,2211320B-1,A,Precinct 20B-1,Precinct 20B-1,...,None,None,20B-1,None,None,None,663,0.001440,0.230453,"POLYGON ((-92.22543 29.96176, -92.22660 29.961..."
8,2100,4.070969,3394,22,113,20A-2,2211320A-2,A,Precinct 20A-2,Precinct 20A-2,...,None,None,20A-2,None,None,None,369,0.000990,0.178493,"POLYGON ((-92.24185 29.96243, -92.24242 29.962..."
9,3602,30.868038,3357,22,111,20,2211120,A,Precinct 20,Precinct 20,...,None,None,20,None,None,None,587,0.007680,0.590152,"POLYGON ((-92.29989 32.68925, -92.30001 32.689..."
